In [ ]:
# Install PySpark
!pip install -q pyspark



In [ ]:
# Create Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BankingCustomerChurn") \
    .getOrCreate()

spark


In [ ]:
from google.colab import files
uploaded = files.upload()  # click "Choose Files" and upload your CSV, e.g., Churn_Modelling.csv

# After upload, list files to confirm
import os
print("Files in /content/:")
print(os.listdir("/content"))


In [ ]:
# Change filename if different
csv_path = "/content/Churn_Modelling.csv"

df = spark.read.option("header", "true") \
               .option("inferSchema", "true") \
               .csv(csv_path)

# Basic checks
print("Number of rows:", df.count())
print("Columns:", df.columns)
df.printSchema()
df.show(5)


In [ ]:
from pyspark.sql.functions import col

# Drop columns that are identifiers or not useful
cols_to_drop = ["RowNumber", "CustomerId", "Surname"]
df_clean = df.drop(*cols_to_drop)

# Verify
print("Columns after dropping:", df_clean.columns)
df_clean.groupBy("Exited").count().show()


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

categorical_cols = ["Geography", "Gender"]
numeric_cols = ["CreditScore", "Age", "Tenure", "Balance",
                "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]

label_col = "Exited"

indexers = [
    StringIndexer(inputCol=c, outputCol=c + "_index", handleInvalid="keep")
    for c in categorical_cols
]

encoders = [
    OneHotEncoder(inputCols=[c + "_index"], outputCols=[c + "_ohe"])
    for c in categorical_cols
]

assembler_inputs = numeric_cols + [c + "_ohe" for c in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")


In [ ]:
train_df, test_df = df_clean.randomSplit([0.8, 0.2], seed=42)

print("Train rows:", train_df.count())
print("Test rows:", test_df.count())



In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(featuresCol="features", labelCol=label_col, maxIter=20)

pipeline_lr = Pipeline(stages=indexers + encoders + [assembler, lr])
lr_model = pipeline_lr.fit(train_df)

lr_predictions = lr_model.transform(test_df)


In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import when

evaluator = BinaryClassificationEvaluator(labelCol=label_col, metricName="areaUnderROC")

lr_auc = evaluator.evaluate(lr_predictions)
print("Logistic Regression AUC:", lr_auc)


In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="features", labelCol=label_col,
                            numTrees=100, maxDepth=6, seed=42)

pipeline_rf = Pipeline(stages=indexers + encoders + [assembler, rf])
rf_model = pipeline_rf.fit(train_df)

rf_predictions = rf_model.transform(test_df)

rf_auc = evaluator.evaluate(rf_predictions)
print("Random Forest AUC:", rf_auc)


In [ ]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(featuresCol="features", labelCol=label_col,
                    maxIter=50, maxDepth=5, stepSize=0.1, seed=42)

pipeline_gbt = Pipeline(stages=indexers + encoders + [assembler, gbt])
gbt_model = pipeline_gbt.fit(train_df)

gbt_predictions = gbt_model.transform(test_df)

gbt_auc = evaluator.evaluate(gbt_predictions)
print("GBT AUC:", gbt_auc)


In [ ]:
print("Model Comparison:")
print("LR AUC:", lr_auc)
print("RF AUC:", rf_auc)
print("GBT AUC:", gbt_auc)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

gbt_pd = gbt_predictions.select("prediction", "Exited").toPandas()

y_true = gbt_pd["Exited"]
y_pred = gbt_pd["prediction"]

cm = confusion_matrix(y_true, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("GBT Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

gbt_pd = gbt_predictions.select("rawPrediction", "Exited").toPandas()
gbt_pd["score"] = gbt_pd["rawPrediction"].apply(lambda x: float(x[1]))

y_true = gbt_pd["Exited"]
y_scores = gbt_pd["score"]

fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(7,6))
plt.plot(fpr, tpr, label=f'GBT ROC (AUC = {roc_auc:.4f})')
plt.plot([0,1],[0,1],'--')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("GBT ROC Curve")
plt.legend()
plt.show()


In [ ]:
rf_stage_model = rf_model.stages[-1]
importances = rf_stage_model.featureImportances

feature_importance = list(zip(assembler_inputs, importances.toArray()))
feature_importance_sorted = sorted(feature_importance, key=lambda x: x[1], reverse=True)

print("Feature Importances:")
for name, score in feature_importance_sorted:
    print(f"{name}: {score}")


## Conclusion

In this project, multiple machine learning models were trained using PySpark to predict bank customer churn.

Among all models, Gradient Boosted Trees (GBT) achieved the highest AUC score, making it the best-performing model.

The model can help banks identify customers who are likely to churn and take proactive steps to improve retention.
